# Portfolio Trader

In [1]:
import ccxt
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

In [2]:
pd.set_option('display.float_format', lambda x: '%5f' % x)

In [3]:
load_dotenv()

True

In [4]:
api_key = os.getenv('BINANCE_TESTNET_API_KEY')
secret_key = os.getenv('BINANCE_TESTNET_API_SECRET')
portfolio_path = os.getenv('PORTFOLIO_PATH')

In [5]:
class PortfolioTrader():
    def __init__(self, exchange):
        self.exchange = exchange

    def __repr__(self):
        return 'PortfolioTrader(exchange={})'.format(self.exchange)

    def load_target_weights(self, filepath, weights_tolerance):
        self.filepath = filepath
        self.target_weights = pd.read_csv(filepath, usecols=[0,1])
        self.target_weights.columns = ['asset', 't_weights']
        self.target_weights.set_index('asset', inplace=True)
        if abs(self.target_weights.t_weights.sum() - 1 > weights_tolerance):
            print('Weights don\'t sum up to 1.')

In [6]:
exchange = ccxt.binance()
exchange

ccxt.binance()

In [7]:
exchange.set_sandbox_mode(True)
exchange.apiKey = api_key
exchange.secret = secret_key

In [8]:
pt = PortfolioTrader(exchange=exchange)
pt

PortfolioTrader(exchange=Binance)

In [9]:
portfolio_path

'C:\\Users\\xjayc\\PycharmProjects\\pythonDataCookbook\\data\\Portfolio.csv'

In [10]:
pt.load_target_weights(filepath=portfolio_path, weights_tolerance=0.01)

In [11]:
pt.target_weights

,t_weights
asset,
USDT,0.500000
BTC,0.100000
ETH,0.200000
SOL,0.200000


In [13]:
class PortfolioTrader():
    def __init__(self, exchange):
        self.exchange = exchange

    def __repr__(self):
        return 'PortfolioTrader(exchange={})'.format(self.exchange)

    def load_target_weights(self, filepath, weights_tolerance):
        self.filepath = filepath
        self.target_weights = pd.read_csv(filepath, usecols=[0,1])
        self.target_weights.columns = ['asset', 't_weights']
        self.target_weights.set_index('asset', inplace=True)
        if abs(self.target_weights.t_weights.sum() - 1 > weights_tolerance):
            print('Weights don\'t sum up to 1.')

    def load_raw_balances(self):
        self.raw_balances = self.exchange.fetch_balance()['info']['balances']
        df = pd.DataFrame(self.raw_balances)
        df = df[['asset','free']].copy()
        df.free = pd.to_numeric(df.free)
        df.set_index('asset', inplace=True)
        df = pd.merge(left=df, right=self.target_weights, how='outer', on='asset')
        df.fillna(0, inplace=True)
        df['ticker'] = np.nan
        df['usdt_price'] = np.nan
        df['usdt'] = np.nan
        df.loc[(df.free != 0) | (df.t_weights !=0)]
        self.rebalance_table = df.copy()

In [14]:
pt = PortfolioTrader(exchange=exchange)

In [15]:
pt.load_target_weights(filepath=portfolio_path, weights_tolerance=0.01)
pt.target_weights

,t_weights
asset,
USDT,0.500000
BTC,0.100000
ETH,0.200000
SOL,0.200000


In [16]:
pt.exchange.fetch_balance()['info']['balances']

[{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'},
 {'asset': 'BTC', 'free': '1.00017500', 'locked': '0.00000000'},
 {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
 {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
 {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
 {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
 {'asset': 'USDT', 'free': '10000.41365706', 'locked': '0.00000000'},
 {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}]

In [17]:
pt.load_raw_balances()

In [18]:
pt.rebalance_table

,free,t_weights,ticker,usdt_price,usdt
asset,,,,,
BNB,1000.000000,0.000000,NaN,NaN,NaN
BTC,1.000175,0.100000,NaN,NaN,NaN
BUSD,10000.000000,0.000000,NaN,NaN,NaN
ETH,100.000000,0.200000,NaN,NaN,NaN
LTC,500.000000,0.000000,NaN,NaN,NaN
TRX,500000.000000,0.000000,NaN,NaN,NaN
USDT,10000.413657,0.500000,NaN,NaN,NaN
XRP,50000.000000,0.000000,NaN,NaN,NaN
SOL,0.000000,0.200000,NaN,NaN,NaN


In [40]:
class PortfolioTrader():
    def __init__(self, exchange):
        self.exchange = exchange

    def __repr__(self):
        return 'PortfolioTrader(exchange={})'.format(self.exchange)

    def load_target_weights(self, filepath, weights_tolerance):
        self.filepath = filepath
        self.target_weights = pd.read_csv(filepath, usecols=[0,1])
        self.target_weights.columns = ['asset', 't_weights']
        self.target_weights.set_index('asset', inplace=True)
        if abs(self.target_weights.t_weights.sum() - 1 > weights_tolerance):
            print('Weights don\'t sum up to 1.')

    def load_raw_balances(self):
        self.raw_balances = self.exchange.fetch_balance()['info']['balances']
        df = pd.DataFrame(self.raw_balances)
        df = df[['asset','free']].copy()
        df.free = pd.to_numeric(df.free)
        df.set_index('asset', inplace=True)
        df = pd.merge(left=df, right=self.target_weights, how='outer', on='asset')
        df.fillna(0, inplace=True)
        df['ticker'] = np.nan
        df['usdt_price'] = np.nan
        df['usdt'] = np.nan
        df.loc[(df.free != 0) | (df.t_weights !=0)]
        self.rebalance_table = df.copy()

    def convert_raw_balances(self):
        for symbol in self.rebalance_table.index:
            balance = self.rebalance_table.loc[symbol, 'free']
            if symbol == 'USDT':
                self.rebalance_table.loc[symbol, 'usdt_price'] = 1
            else:
                try:
                    target = 'USDT'
                    self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                except Exception as e:
                    try:
                        target = 'BTC'
                        self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                        self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                    except:
                        try:
                            target = 'ETH'
                            self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                            self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                        except:
                            try:
                                target = 'BNB'
                                self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                                self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                            except:
                                try:
                                    target = 'BUSD'
                                    self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                                    self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                                except:
                                    print('{} can not be converted!'.format(symbol))
        self.rebalance_table['usdt'] = self.rebalance_table.free.mul(self.rebalance_table.usdt_price)
        self.balance = self.rebalance_table['usdt'].sum()

    def _fetch_and_convert(self, symbol, target, orig_symbol, step):
        try:
            ticker = symbol + '/' + target # symbol is base currency
            price = float(self.exchange.fetchTicker(ticker)['last'])
        except:
            ticker = target + '/' + symbol # symbol is quote currency
            price = 1/float(self.exchange.fetchTicker(ticker)['last'])

        if step == 'first':
            self.rebalance_table.loc[symbol, 'ticker'] = ticker
            self.rebalance_table.loc[symbol, 'ticker_price'] = price
            self.rebalance_table.loc[symbol, 'usdt_price'] = price
        elif step == 'second':
            self.rebalance_table.loc[orig_symbol, 'usdt_price'] *= price

In [41]:
pt = PortfolioTrader(exchange=exchange)
pt

PortfolioTrader(exchange=Binance)

In [42]:
pt.load_target_weights(filepath=portfolio_path, weights_tolerance=0.01)

In [43]:
pt.load_raw_balances()

In [44]:
pt.convert_raw_balances()

BUSD can not be converted!
SOL can not be converted!


In [45]:
pt.rebalance_table

,free,t_weights,ticker,usdt_price,usdt,ticker_price
asset,,,,,,
BNB,1000.000000,0.000000,BNB/USDT,307.700000,307700.000000,307.700000
BTC,1.000175,0.100000,BTC/USDT,23193.850000,23197.908924,23193.850000
BUSD,10000.000000,0.000000,BNB/BUSD,0.003249,32.488629,0.003249
ETH,100.000000,0.200000,ETH/USDT,1617.990000,161799.000000,1617.990000
LTC,500.000000,0.000000,LTC/USDT,89.350000,44675.000000,89.350000
TRX,500000.000000,0.000000,TRX/USDT,0.061960,30980.000000,0.061960
USDT,10000.413657,0.500000,NaN,1.000000,10000.413657,NaN
XRP,50000.000000,0.000000,XRP/USDT,0.413800,20690.000000,0.413800
SOL,0.000000,0.200000,NaN,NaN,NaN,NaN
